
<h3 align="center">به نام خدا </h3>
<h2 align="center">تمرین سری چهارم </h2>


<p align="right">در این تمرین می خواهیم با کمک دانسته های خود از توزیع نرمال یک متغیره یک دسته بند طراحی کنیم</p>

<p align="right">برای پیاده سازی مدل ها از کتابخانه  استفاده کنید scikitlearn</p>

<p align="right">https://scikit-learn.org/stable/</p>

# خواندن داده ها

<p align="right">ابتدا داده ها را از کتابخانه می خوانیم و سپس داده ها را به دو دسته ی آموزش و آزمایش تقسیم بندی می کنیم</p>

In [1]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

# store the feature matrix (X) and response vector (y) 
X = iris.data
y = iris.target
print("our dataset has " + str(X.shape[1]) + " features. for more information about data surf the web")
# splitting X and y into training and testing sets
#you can change the test size, fit model with more or less data and see results
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1) 

import numpy as np
from math import pi

our dataset has 4 features. for more information about data surf the web


In [2]:
X[0]

array([5.1, 3.5, 1.4, 0.2])

### q1) print the number of train and test data and number of classes

In [3]:
print("the number of train data is : {}".format(len(X_train)) )
print("the number of test data is {}: ".format(len(X_test)) )
print("there is {} different classes in the dataset".format(len(set(y))) )

the number of train data is : 90
the number of test data is 60: 
there is 3 different classes in the dataset


# آموزش یک مدل با فرض گوسی بودن توزیع داد ها

<p align="right">فرض کنید می دانیم که داده های ما از یک توزیع گوسی پیروی می کنند و همچنین ویژگی ها از هم مستقل هستند لطفا با کمک فرض های مساله و بدون کمک از مدل های آماده یک مدل برای دسته بندی کلاس ها آموزش دهید . دقت کنید که نباید از کدهای آماده استفاده کنید و فقط مجاز هستید برای فرمول های پایه مثل میانگین یا واریانس از توابع آماده استفاده نمایید</p>

In [4]:
x_c0 = np.array([ X_train[i] for i in range( len(y_train) ) if y_train[i] == 0 ])
x_c1 = np.array([ X_train[i] for i in range( len(y_train) ) if y_train[i] == 1 ])
x_c2 = np.array([ X_train[i] for i in range( len(y_train) ) if y_train[i] == 2 ])

In [5]:
#calculate mean and variance.
def get_mean_var(df):
    features_count = df.shape[1]
    mean , var = np.zeros(features_count) , np.zeros(features_count)
    for i in range(features_count):
        mean[i] = df.transpose()[i].mean()
        var[i]  = df.transpose()[i].var()
    return mean, var

In [6]:
mean_c0, var_c0 = get_mean_var(x_c0)
mean_c1, var_c1 = get_mean_var(x_c1)
mean_c2, var_c2 = get_mean_var(x_c2)

In [7]:
#calculate P(Ci)
p_c0 = len(x_c0) / len(y_train)
p_c1 = len(x_c1) / len(y_train)
p_c2 = len(x_c2) / len(y_train)

In [8]:
def calc_p(X,mean,var):
    return (-np.log(np.sqrt(var))-((X-mean)**2)/(2*var))

In [9]:
p = np.zeros((4,len(X_test)))

for i in range(len(X_test)):
    temp_0 =  calc_p( X_test[i,0], mean_c0[0], var_c0[0] ) \
            + calc_p( X_test[i,1], mean_c0[1], var_c0[1] ) \
            + calc_p( X_test[i,2], mean_c0[2], var_c0[2] ) \
            + calc_p( X_test[i,3], mean_c0[3], var_c0[3] )
    
    G_0 = -.5*np.log(2*pi) + temp_0 + np.log(p_c0)

    temp_1 =  calc_p( X_test[i,0], mean_c1[0], var_c1[0] ) \
            + calc_p( X_test[i,1], mean_c1[1], var_c1[1] ) \
            + calc_p( X_test[i,2], mean_c1[2], var_c1[2] ) \
            + calc_p( X_test[i,3], mean_c1[3], var_c1[3] )
        
    G_1 = -.5*np.log(2*pi) + temp_1 + np.log(p_c1)

    temp_2 =  calc_p( X_test[i,0], mean_c2[0], var_c2[0] ) \
            + calc_p( X_test[i,1], mean_c2[1], var_c2[1] ) \
            + calc_p( X_test[i,2], mean_c2[2], var_c2[2] ) \
            + calc_p( X_test[i,3], mean_c2[3], var_c2[3] )
    
    G_2 = -.5*np.log(2*pi) + temp_2 + np.log(p_c2)
    
    g_0 = np.exp(G_0)
    g_1 = np.exp(G_1)
    g_2 = np.exp(G_2)
    
    p[0][i] = g_0/(g_1+g_0+g_2)
    p[1][i] = g_1/(g_1+g_0+g_2)
    p[2][i] = g_2/(g_1+g_0+g_2)
    
    p[3][i] = np.argmax([p[0][i], p[1][i], p[2][i]])

<p align="right">اکنون با کمک داده های آزمایشی دقت مدل خود را بسنجید</p>

In [10]:
y_pred_GNB = np.array(1 *(p[3]==y_test))
print('Accuracy =',(np.count_nonzero(y_pred_GNB)/len(y_pred_GNB)))

Accuracy = 0.95


# آموزش مدل بدون دانستن توزیع داده ها

<p align="right">داده ها را با کمک دسته بند اس وی ام و همچنین یک شبکه عصبی ساده دسته بندی کنید و دقت را با بخش قبل مقایسه نمایید</p>

In [11]:
from sklearn import svm
from sklearn import metrics

#train svm model
#write yor code here :

clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy: {}%".format( round( metrics.accuracy_score(y_test, y_pred) * 100 , 2 ) ))

Accuracy: 98.33%


In [12]:
from sklearn.neural_network import MLPClassifier
#use two hidden layers 
#train multi layer perceptron
#write yor code here :

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(20,20), max_iter=5000)
mlp.fit(X_train,y_train)
y_pred = mlp.predict(X_test)

print("Accuracy: {}%".format( round( metrics.accuracy_score(y_test, y_pred) * 100 , 2 ) ))

Accuracy: 100.0%




---



# خواندن داده های جدید

In [13]:
wine = datasets.load_wine()
# store the feature matrix (X) and response vector (y) 
X = wine.data 
y = wine.target 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1) 

# آموزش یک مدل با فرض گوسی بودن توزیع داد ها

<p align="right">فرض کنید می دانیم که داده های ما از یک توزیع گوسی پیروی می کنند با کمک کتابخانه های آماده یک مدل برای دسته بندی کلاس ها آموزش دهید می  توانید برای این قسمت از کتابخانه های آماده نیز استفاده کنید و نیازی به پیاده سازی نیست</p>

In [14]:
from sklearn.naive_bayes import GaussianNB
# training the model on training set 

# write your code here :

clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

<p align="right">اکنون با کمک داده های آزمایشی دقت مدل خود را بسنجید</p>

In [15]:
#write your code here :
print("Accuracy: {}%".format( round( metrics.accuracy_score(y_test, y_pred) * 100 , 2 ) ))

Accuracy: 98.61%


# آموزش مدل بدون دانستن توزیع داده ها

<p align="right">داده ها را با کمک دسته بند اس وی ام و همچنین یک شبکه عصبی ساده دسته بندی کنید و دقت را با بخش قبل مقایسه نمایید</p>

In [16]:
#train svm model

#write yor code here :

from sklearn.svm import SVC
from sklearn import metrics

clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy: {}%".format( round( metrics.accuracy_score(y_test, y_pred) * 100 , 2 ) ))

Accuracy: 95.83%


In [17]:
#train multi layer perceptron

#write yor code here

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(20,20), max_iter=5000)
mlp.fit(X_train,y_train)
y_pred = mlp.predict(X_test)

print("Accuracy: {}%".format( round( metrics.accuracy_score(y_test, y_pred) * 100 , 2 ) ))

Accuracy: 97.22%




---



#بخش امتیازی

<p align="right">آیا راهی وجود دارد که قبل از انتخاب بک دسته بند بتوان توزیع یک داده را حدس زد و دسته بندی را بر اساس این بینش از داده ها انجام داد؟</p>

### توضیحات این بخش در فایل گزارش تمرین آورده شده است.

# Question 3

In [18]:
import numpy as np

In [19]:
def estimator(a,b,degree,offset = 0) :
    c = np.zeros([degree,degree])
    for i in range(degree):
        for j in range(degree):
            g = i + j
            c[i][j] = sum([(i-offset)**g for i in a])
    c2 = np.linalg.inv(c)
    d = np.zeros(degree)
    for i in range(degree) :
        d[i] = sum( [ b[j] * (a[j]-offset) ** i for j in range(len(a)) ] )
    return c,d, np.matmul(c2,d)

In [20]:
X = [ 1394, 1395, 1396,1397, 1398 ]
Y = [12 ,19 ,29, 37 ,45]

In [21]:
A,y,w = estimator(X,Y,3)

In [22]:
print("A : \n",A)
print("y : \n",y)
print("w : \n",w)

A : 
 [[5.00000000e+00 6.98000000e+03 9.74409000e+06]
 [6.98000000e+03 9.74409000e+06 1.36027776e+10]
 [9.74409000e+06 1.36027776e+10 1.89895359e+13]]
y : 
 [1.42000000e+02 1.98316000e+05 2.76966684e+08]
w : 
 [-1.16969531e+04  8.39851379e+00  5.21540642e-07]


In [23]:
p1 = np.polyval(np.flip(w, 0), 1399)
print("Estimation for 1399 : {}".format(round(p1,2)))

Estimation for 1399 : 53.59
